# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [47]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
import pickle
from custom_transformer import StartingVerbExtractor

ModuleNotFoundError: No module named 'custom_transformer'

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)
X = df.message
Y = df.iloc[:,4:]

In [3]:
#https://knowledge.udacity.com/questions/726905

df.related.unique()

array([1, 0, 2])

In [4]:
for column in Y.columns:
    print(column,df[column].unique())

related [1 0 2]
request [0 1]
offer [0 1]
aid_related [0 1]
medical_help [0 1]
medical_products [0 1]
search_and_rescue [0 1]
security [0 1]
military [0 1]
child_alone [0]
water [0 1]
food [0 1]
shelter [0 1]
clothing [0 1]
money [0 1]
missing_people [0 1]
refugees [0 1]
death [0 1]
other_aid [0 1]
infrastructure_related [0 1]
transport [0 1]
buildings [0 1]
electricity [0 1]
tools [0 1]
hospitals [0 1]
shops [0 1]
aid_centers [0 1]
other_infrastructure [0 1]
weather_related [0 1]
floods [0 1]
storm [0 1]
fire [0 1]
earthquake [0 1]
cold [0 1]
other_weather [0 1]
direct_report [0 1]


In [5]:
df=df[df.related!=2]

In [6]:
df.shape

(26028, 40)

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens if word not in stopwords.words("english")]
    return tokens

In [39]:
import nltk
nltk.download(["punkt","stopwords","wordnet","averaged_perceptron_tagger"])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = pipeline.predict(X_test)

In [19]:
for i, c in enumerate(Y.columns): 
    print(c) 
    print(classification_report(np.array(y_test)[i], y_pred[i]))

related
             precision    recall  f1-score   support

          0       0.91      1.00      0.95        31
          1       1.00      0.40      0.57         5

avg / total       0.92      0.92      0.90        36

request
             precision    recall  f1-score   support

          0       0.97      1.00      0.99        33
          1       1.00      0.67      0.80         3

avg / total       0.97      0.97      0.97        36

offer
             precision    recall  f1-score   support

          0       0.77      1.00      0.87        27
          1       1.00      0.11      0.20         9

avg / total       0.83      0.78      0.70        36

aid_related
             precision    recall  f1-score   support

          0       0.91      1.00      0.95        31
          1       1.00      0.40      0.57         5

avg / total       0.92      0.92      0.90        36

medical_help
             precision    recall  f1-score   support

          0       0.97      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
parameters = {
    'clf__estimator__n_estimators': [5,10,15],
    'clf__estimator__min_samples_split': [2,5]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
cv.fit(X_train,y_train)
y_pred_update = cv.predict(X_test)
for i, c in enumerate(Y.columns): 
    print(c) 
    print(classification_report(np.array(y_test)[i], y_pred_update[i]))

related
             precision    recall  f1-score   support

          0       0.91      1.00      0.95        31
          1       1.00      0.40      0.57         5

avg / total       0.92      0.92      0.90        36

request
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        33
          1       1.00      1.00      1.00         3

avg / total       1.00      1.00      1.00        36

offer
             precision    recall  f1-score   support

          0       0.77      1.00      0.87        27
          1       1.00      0.11      0.20         9

avg / total       0.83      0.78      0.70        36

aid_related
             precision    recall  f1-score   support

          0       0.89      1.00      0.94        31
          1       1.00      0.20      0.33         5

avg / total       0.90      0.89      0.86        36

medical_help
             precision    recall  f1-score   support

          0       0.97      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [36]:
from sklearn.base import BaseEstimator, TransformerMixin
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list =nltk.sent_tokenize(text)        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(sentence)
            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
            return False
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [45]:
pipeline_update= Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor())
    ])),
    ('clf', RandomForestClassifier())
        ])
pipeline_update.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [46]:
y_pred_update2 = pipeline_update.predict(X_test)

TypeError: no supported conversion for types: (dtype('float64'), dtype('O'))

In [ ]:
for i, c in enumerate(Y.columns): 
    print(c) 
    print(classification_report(np.array(y_test)[i], y_pred_update2[i]))

### 9. Export your model as a pickle file

In [43]:
pickle.dump(pipeline_update, open('final_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.